### 1. Imports 

In [1]:
import pandas as pd
import numpy as np
import json
import os

from sagemaker.serializers import CSVSerializer
from sagemaker.inputs import TrainingInput
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role
import sagemaker
import boto3

In [2]:
sagemaker.__version__

'2.15.2'

### 2. Essentials 

In [3]:
# Get the Execution Role for this Notebook and AWS Session to create Clients
sagemaker_execution_role = get_execution_role()
print('Role = {}'.format(sagemaker_execution_role))
session = boto3.Session()

# Clients and Resources
s3 = session.resource('s3')
sagemaker_session = sagemaker.Session()

BUCKET = 'sagemaker-demo-892313895307' # USE YOUR ACCOUNT ID OR INITIALS AS SUFFIX
PREFIX = 'clf'

Role = arn:aws:iam::892313895307:role/service-role/AmazonSageMaker-ExecutionRole-20200826T084395


##### Upload Train & Test Sets to S3 and Create Pointers to Data

In [4]:
s3.create_bucket(Bucket=BUCKET)

s3.Bucket(name='sagemaker-demo-892313895307')

In [5]:
s3.Bucket(BUCKET).Object(os.path.join(PREFIX, 'train/train.csv')).upload_file('./DATA/train/train.csv')
s3.Bucket(BUCKET).Object(os.path.join(PREFIX, 'test/test.csv')).upload_file('./DATA/test/test.csv')
s3.Bucket(BUCKET).Object(os.path.join(PREFIX, 'batch_test/batch_test.csv')).upload_file('./DATA/batch_test/batch_test.csv')

##### Create Pointers to the uploaded files 

In [6]:
train_set_location = 's3://{}/{}/train/'.format(BUCKET, PREFIX)
test_set_location = 's3://{}/{}/test/'.format(BUCKET, PREFIX)

In [7]:
print(train_set_location)
print(test_set_location)

s3://sagemaker-demo-892313895307/clf/train/
s3://sagemaker-demo-892313895307/clf/test/


In [8]:
train_set_pointer = TrainingInput(s3_data=train_set_location, content_type='csv')
test_set_pointer = TrainingInput(s3_data=test_set_location, content_type='csv')

In [9]:
print(json.dumps(train_set_pointer.__dict__, indent=2))

{
  "config": {
    "DataSource": {
      "S3DataSource": {
        "S3DataType": "S3Prefix",
        "S3Uri": "s3://sagemaker-demo-892313895307/clf/train/",
        "S3DataDistributionType": "FullyReplicated"
      }
    },
    "ContentType": "csv"
  }
}


### 3. Train a Model using SageMaker + Builtin XgBoost Algorithm

In [10]:
container_uri = sagemaker.image_uris.retrieve(region=session.region_name, 
                                              framework='xgboost', 
                                              version='1.0-1', 
                                              image_scope='training')

In [11]:
xgb = sagemaker.estimator.Estimator(image_uri=container_uri,
                                    role=sagemaker_execution_role, 
                                    instance_count=2, 
                                    instance_type='ml.m5.large',
                                    output_path='s3://{}/{}/model-artifacts'.format(BUCKET, PREFIX),
                                    sagemaker_session=sagemaker_session,
                                    base_job_name='classifier')

xgb.set_hyperparameters(objective='binary:logistic',
                        num_round=100)

In [12]:
xgb.fit({'train': train_set_pointer, 'validation': test_set_pointer})

2020-11-09 02:18:10 Starting - Starting the training job...
2020-11-09 02:18:17 Starting - Launching requested ML instances.........
2020-11-09 02:19:48 Starting - Preparing the instances for training...
2020-11-09 02:20:42 Downloading - Downloading input data......
2020-11-09 02:21:20 Training - Downloading the training image.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[02:21:36] 7461x4 matrix with 29844 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Determined delimiter of

### 4. Host the Trained Model as a SageMaker Endpoint (using Estimator object)

In [13]:
xgb_predictor = xgb.deploy(initial_instance_count=2,
                           instance_type='ml.m5.large')

---------------!

### 5. Real Time Inference from the Endpoint

In [14]:
csv_serializer = CSVSerializer()
predictor = Predictor(endpoint_name=xgb_predictor.endpoint_name, 
                      serializer=csv_serializer)

In [15]:
test_df = pd.read_csv('./DATA/test/test.csv', names=['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])

In [16]:
test_df.head()

,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
0,0,0.132043,0.088718,0.047720,0.00000
1,0,0.217554,0.050000,0.078014,0.01942
2,0,0.526601,0.200000,0.085106,0.00000
3,1,0.191450,0.111282,0.111550,0.00000
4,1,0.212498,0.450000,0.170213,0.04283


In [17]:
X = test_df.sample(1)
X

,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
1532,0,0.523235,0.025,0.028369,0.0


In [18]:
X = X.values[0]
X[1:]

array([0.52323543, 0.025     , 0.02836879, 0.        ])

In [19]:
payload = X[1:]

In [20]:
%%time

predicted_class_prob = predictor.predict(payload).decode('utf-8')
if float(predicted_class_prob) < 0.5:
    print('PREDICTION = NOT DIABETIC')
else:
    print('PREDICTION = DIABETIC')
print()

PREDICTION = NOT DIABETIC

CPU times: user 11.2 ms, sys: 0 ns, total: 11.2 ms
Wall time: 151 ms


### 5. Evaluate Hosted Model for Accuracy

In [21]:
predictions = []
expected = []
correct = 0
for row in test_df.values:
    expected_class = row[0]
    payload = row[1:]
    predicted_class_prob = predictor.predict(payload).decode('utf-8')
    
    predicted_class = 1
    if float(predicted_class_prob) < 0.5:
        predicted_class = 0
        
    if predicted_class == expected_class:
        correct += 1
    predictions.append(predicted_class)
    expected.append(expected_class)

In [22]:
print('Accuracy = {:.2f}%'.format(correct/len(predictions) * 100))

Accuracy = 77.56%


#### Confusion Matrix

In [23]:
expected = pd.Series(np.array(expected))
predictions = pd.Series(np.array(predictions))
pd.crosstab(expected, predictions, rownames=['Actual'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
Actual,,,
0.0,1908,72,1980
1.0,486,21,507
All,2394,93,2487
